<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MISTRAL_AAI_DEMO_JULY2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mistralai -q
!pip install colab-env -q

In [7]:
import os
import time
import random
import json
import colab_env # Uncomment if you are in Google Colab (if applicable)

# It's good practice to get the API key safely and handle its absence
api_key = os.environ.get("MISTRAL_API_KEY")

if not api_key:
    print("Error: MISTRAL_API_KEY environment variable not set.")
    print("Please set your Mistral API key before running this script.")
    exit()

try:
    from mistralai import Mistral # Correct import as per your reference
except ImportError:
    print("Please install the Mistral AI library: pip install mistralai")
    exit()

# Initialize client using the Mistral class
client = Mistral(api_key=api_key)

# Test to ensure client works and list models
try:
    model_list = client.models.list()
    print("Available Mistral Models:")
    # Print all model IDs
    for model_info in model_list.data:
        print(model_info.id)

    MISTRAL_MODEL = "magistral-medium-latest"
    print(f"\nUsing Mistral model: {MISTRAL_MODEL}")
except Exception as e:
    print(f"Error connecting to Mistral API or listing models: {e}")
    print("Please check your API key and network connection.")
    exit()


Available Mistral Models:
mistral-medium-2505
mistral-medium-latest
mistral-medium
ministral-3b-2410
ministral-3b-latest
ministral-8b-2410
ministral-8b-latest
open-mistral-7b
mistral-tiny
mistral-tiny-2312
open-mistral-nemo
open-mistral-nemo-2407
mistral-tiny-2407
mistral-tiny-latest
open-mixtral-8x7b
mistral-small
mistral-small-2312
open-mixtral-8x22b
open-mixtral-8x22b-2404
mistral-small-2409
mistral-large-2407
mistral-large-2411
mistral-large-latest
pixtral-large-2411
pixtral-large-latest
mistral-large-pixtral-2411
codestral-2501
codestral-latest
codestral-2412
codestral-2411-rc5
devstral-small-2505
devstral-small-latest
pixtral-12b-2409
pixtral-12b
pixtral-12b-latest
mistral-small-2501
mistral-small-2503
mistral-small-2506
mistral-small-latest
mistral-saba-2502
mistral-saba-latest
magistral-medium-2506
magistral-medium-latest
magistral-small-2506
magistral-small-latest
mistral-embed
codestral-embed
codestral-embed-2505
mistral-moderation-2411
mistral-moderation-latest
mistral-ocr-2

In [9]:
# --- 1. Environment Simulation ---
class FlightEnvironment:
    def __init__(self, initial_weather="clear", initial_traffic="low"):
        self.weather = initial_weather
        self.traffic = initial_traffic
        self.current_location = "Montreal"
        self.destination = "New York"
        self.flight_status = "idle" # idle, planning, in-flight, landed
        self.fuel_level = 100 # percentage
        self.time_elapsed = 0
        self.active_flight_plan = None # Store a more detailed plan generated by LLM

    def update_environment(self):
        """Simulates changes in weather and traffic."""
        self.time_elapsed += 1
        # Environmental changes for testing specific scenarios
        # These are now set within the run_test_case for specific steps.
        # This function still handles general time progression and fuel consumption.

        # Simulate fuel consumption if in-flight
        if self.flight_status == "in-flight":
            self.fuel_level -= random.randint(1, 3) # Simulate consumption
            if self.fuel_level <= 0:
                print("Environment: Out of fuel! Emergency landing initiated.")
                self.flight_status = "landed" # For simulation simplicity

        print(f"Environment Status (Time: {self.time_elapsed}): Weather: {self.weather}, Traffic: {self.traffic}, Fuel: {self.fuel_level}%")

    def get_observation(self):
        """Provides the agent with the current state of the environment."""
        return {
            "current_location": self.current_location,
            "destination": self.destination,
            "weather": self.weather,
            "traffic": self.traffic,
            "flight_status": self.flight_status,
            "fuel_level": self.fuel_level,
            "time_elapsed": self.time_elapsed,
            "active_flight_plan": self.active_flight_plan # Include plan in observation
        }

# --- 2. Agent Core ---
class FlightPlanningAgent:
    def __init__(self, mistral_client, mistral_model, name="Mistral_Flight_Agent_2.0_LLM"):
        self.name = name
        self.mistral_client = mistral_client
        self.mistral_model = mistral_model
        self.goal = "Successfully fly from Montreal to New York."
        self.memory = [] # To store past observations and LLM interactions

        # Tools the agent can 'use' (simulate external functions)
        self.tools = {
            "plan_optimal_route": self._plan_optimal_route,
            "check_weather_forecast": self._check_weather_forecast,
            "check_air_traffic": self._check_air_traffic,
            "initiate_takeoff": self._initiate_takeoff,
            "proceed_en_route": self._proceed_en_route,
            "initiate_landing": self._initiate_landing,
            "replan_flight_due_to_conditions": self._replan_flight_due_to_conditions,
            "wait_for_conditions": self._wait_for_conditions,
            "report_goal_achieved": self._report_goal_achieved
        }

    def _plan_optimal_route(self, current_location, destination, weather, traffic):
        """Simulates planning an optimal route based on conditions."""
        return f"Simulated: Calculated optimal route from {current_location} to {destination} considering {weather} weather and {traffic} traffic."

    def _check_weather_forecast(self, location):
        """Simulates checking weather forecast."""
        return f"Simulated: Weather forecast for {location} is currently variable."

    def _check_air_traffic(self, location):
        """Simulates checking air traffic control status."""
        return f"Simulated: Air traffic at {location} is moderate."

    def _initiate_takeoff(self):
        return "Simulated: Takeoff sequence initiated."

    def _proceed_en_route(self):
        return "Simulated: Continuing en route."

    def _initiate_landing(self):
        return "Simulated: Landing sequence initiated."

    def _replan_flight_due_to_conditions(self, reason):
        return f"Simulated: Flight replanning initiated due to: {reason}."

    def _wait_for_conditions(self):
        return "Simulated: Agent waiting for conditions to improve."

    def _report_goal_achieved(self):
        return "Simulated: Goal of successful flight achieved!"

    def perceive(self, environment_observation):
        """
        Agent's perception module.
        Gathers information from the environment.
        """
        self.memory.append(environment_observation)
        print(f"\n{self.name} perceives: {environment_observation}")
        return environment_observation

    def reason(self, observation, environment):
        """
        Agent's reasoning and planning module, powered by Mistral AI LLM.
        Formulates a prompt to the LLM based on current observation and past memory.
        The LLM's response guides the next action.
        """
        current_status = observation["flight_status"]
        current_location = observation["current_location"]
        destination = observation["destination"]
        weather = observation["weather"]
        traffic = observation["traffic"]
        fuel = observation["fuel_level"]
        time_elapsed = observation["time_elapsed"]
        active_flight_plan = observation["active_flight_plan"]

        # --- Enhanced System Prompt for Stricter LLM Adherence ---
        prompt_messages = [
            {"role": "system", "content": f"""You are an expert AI Flight Planning Agent named {self.name}. Your ultimate and overarching goal is to "{self.goal}".
You must analyze the current situation **strictly following the prioritized decision rules below**.
Your response MUST be a valid JSON object with an 'action' key and a 'parameters' object.
The 'action' must be one of: {', '.join(self.tools.keys())}, 'report_goal_achieved', or 'no_action'.

**CRITICAL PRIORITIZED DECISION FLOW (Read and Apply from Top to Bottom):**

1.  **IMMEDIATE SAFETY & GOAL CHECK (Highest Priority - Do NOT bypass):**
    * **IF** `flight_status` is 'in-flight' AND `fuel_level` <= 15%:
        * **Action:** `initiate_landing`
        * **Parameters:** `{{}}`
        * **Justification:** IMMEDIATE EMERGENCY: Fuel critically low, must land NOW. This overrides all other considerations.
    * **ELSE IF** `flight_status` is 'landed' AND `current_location` is '{destination}':
        * **Action:** `report_goal_achieved`
        * **Parameters:** `{{}}`
        * **Justification:** Goal completed successfully.

2.  **IN-FLIGHT ADAPTATION (High Priority):**
    * **ELSE IF** `flight_status` is 'in-flight' AND `weather` == 'stormy':
        * **Action:** `replan_flight_due_to_conditions`
        * **Parameters:** `{{'reason': 'severe stormy weather encountered en route'}}`
        * **Justification:** In-flight stormy weather is dangerous; immediate replanning to avoid.
    * **ELSE IF** `flight_status` is 'in-flight' AND `fuel_level` < 30% AND `traffic` == 'high':
        * **Action:** `replan_flight_due_to_conditions`
        * **Parameters:** `{{'reason': 'low fuel combined with high traffic near destination'}}`
        * **Justification:** Dual risk requires urgent re-evaluation of route or diversion.
    * **ELSE IF** `flight_status` is 'in-flight':
        * **Action:** `proceed_en_route`
        * **Parameters:** `{{}}`
        * **Justification:** Continue as planned towards destination.

3.  **PRE-FLIGHT PLANNING & TAKEOFF (Medium Priority):**
    * **ELSE IF** `flight_status` IN ('idle', 'planning') AND `active_flight_plan` IS NULL:
        * **Action:** `plan_optimal_route`
        * **Parameters:** `{{'current_location': '{current_location}', 'destination': '{destination}', 'weather': '{weather}', 'traffic': '{traffic}'}}`
        * **Justification:** No plan exists; must create one to proceed. Also, include `plan_details` in your JSON.
    * **ELSE IF** `flight_status` == 'planning' AND (`weather` == 'stormy' OR `traffic` == 'high'):
        * **Action:** `wait_for_conditions`
        * **Parameters:** `{{'reason': 'unfavorable pre-flight conditions (stormy weather or high traffic)'}}`
        * **Justification:** Safety dictates waiting before takeoff due to current adverse conditions.
    * **ELSE IF** `flight_status` == 'planning' AND `weather` != 'stormy' AND `traffic` != 'high' AND `active_flight_plan` IS NOT NULL:
        * **Action:** `initiate_takeoff`
        * **Parameters:** `{{}}`
        * **Justification:** Conditions are favorable and a flight plan is ready; can proceed with takeoff.

4.  **GENERAL MONITORING (Lowest Priority):**
    * **ELSE:**
        * **Action:** `no_action`
        * **Parameters:** `{{}}`
        * **Justification:** Current state is stable; monitoring for changes or waiting for next command/trigger.

**ALWAYS include 'plan_details' if your action is 'plan_optimal_route'.**
"""},
            {"role": "user", "content": f"Current Situation (Time {time_elapsed}min):\n"
                                               f"  - Flight Status: {current_status}\n"
                                               f"  - Location: {current_location}\n"
                                               f"  - Destination: {destination}\n"
                                               f"  - Weather: {weather}\n"
                                               f"  - Air Traffic: {traffic}\n"
                                               f"  - Fuel Level: {fuel}%\n"
                                               f"  - Current Plan: {active_flight_plan if active_flight_plan else 'None'}\n"}
        ]

        print(f"{self.name} is reasoning using Mistral AI...")
        try:
            # Corrected: Using client.chat.complete as per your last reference
            chat_response = self.mistral_client.chat.complete(
                model=self.mistral_model,
                messages=prompt_messages,
                response_format={"type": "json_object"}, # Request JSON response
                temperature=0.0 # Make it deterministic for decision-making
            )

            llm_response_content = chat_response.choices[0].message.content
            print(f"LLM Raw Response: {llm_response_content}")

            # Parse the LLM's JSON response
            try:
                llm_decision = json.loads(llm_response_content)
                action = llm_decision.get("action", "no_action")
                parameters = llm_decision.get("parameters", {})
                plan_details = llm_decision.get("plan_details", None) # LLM might provide a plan

                # Validate LLM action against known tools and special actions
                valid_actions = set(self.tools.keys()).union({"report_goal_achieved", "no_action"})
                if action not in valid_actions:
                    print(f"Warning: LLM proposed invalid action '{action}'. Falling back to 'no_action'.")
                    action = "no_action"
                    parameters = {}

                # Specific logic to handle 'landed' status for goal check
                # This ensures the agent can always report goal achieved if it successfully landed
                if current_status == "landed" and current_location == destination:
                    if action != "report_goal_achieved":
                        print(f"Agent internally overriding LLM action to 'report_goal_achieved' as goal is met.")
                    return "report_goal_achieved", {} # Override if landed at destination

                # If LLM planned a route, update the environment's active plan
                if action == "plan_optimal_route" and plan_details:
                    environment.active_flight_plan = plan_details

                return action, parameters

            except json.JSONDecodeError as e:
                print(f"Error parsing LLM response JSON: {e}. Raw response: {llm_response_content}")
                return "no_action", {} # Fallback if JSON is malformed

        except Exception as e:
            # THIS IS THE FIXED BLOCK: 'environment' is now accessible.
            print(f"Error calling Mistral AI: {e}")
            # Fallback for API errors - crucial for agent robustness
            # If the API fails, the agent should still attempt to proceed safely or wait
            if current_status == "in-flight" and fuel <= 15:
                print(f"API failed, forcing emergency landing due to critical fuel.")
                return "initiate_landing", {}
            elif current_status == "in-flight" and weather == "stormy":
                print(f"API failed, forcing replan due to stormy weather.")
                return "replan_flight_due_to_conditions", {"reason": "API error and severe weather"}
            else:
                print(f"API failed, falling back to 'no_action' for safety.")
                return "no_action", {}

    def act(self, action_data, environment):
        """
        Agent's action module.
        Performs the decided action, interacting with the environment,
        and mapping LLM-suggested 'tool calls' to simulated functions.
        """
        action = action_data[0]
        params = action_data[1]
        print(f"{self.name} acts: {action} with parameters {params}")

        if action == "plan_optimal_route":
            # Only change status to planning if not already in-flight or landed
            if environment.flight_status in ["idle", "planning"]:
                environment.flight_status = "planning"
            result = self.tools["plan_optimal_route"](
                params.get("current_location", environment.current_location),
                params.get("destination", environment.destination),
                params.get("weather", environment.weather),
                params.get("traffic", environment.traffic)
            )
            print(result)
            if environment.active_flight_plan:
                print(f"LLM-generated plan: {environment.active_flight_plan}")

        elif action == "check_weather_forecast":
            result = self.tools["check_weather_forecast"](params.get("location", environment.current_location))
            print(result)

        elif action == "check_air_traffic":
            result = self.tools["check_air_traffic"](params.get("location", environment.current_location))
            print(result)

        elif action == "initiate_takeoff":
            if environment.flight_status == "planning":
                environment.flight_status = "in-flight"
                print(self.tools["initiate_takeoff"]())
            else:
                print(f"{self.name}: Cannot take off from current status: {environment.flight_status}")

        elif action == "proceed_en_route":
            if environment.flight_status == "in-flight":
                print(self.tools["proceed_en_route"]())
                # Simulate reaching destination after some time
                # This threshold should ideally be based on distance/speed in a real sim
                if environment.time_elapsed > 25 and environment.fuel_level > 20:
                     print(f"{self.name}: Approaching destination for landing...")
            else:
                print(f"{self.name}: Cannot fly, not in-flight.")

        elif action == "initiate_landing":
            if environment.flight_status == "in-flight" or environment.fuel_level <= 0:
                environment.flight_status = "landed"
                environment.current_location = environment.destination # Assume landing at destination
                print(self.tools["initiate_landing"]())
                print(f"{self.name}: Successfully landed at {environment.current_location}.")
            else:
                print(f"{self.name}: Cannot land from current status: {environment.flight_status}")

        elif action == "replan_flight_due_to_conditions":
            environment.active_flight_plan = None # Invalidate current plan
            # Revert to planning state if not already landed
            if environment.flight_status != "landed":
                environment.flight_status = "planning"
            print(self.tools["replan_flight_due_to_conditions"](params.get("reason", "unknown reason")))

        elif action == "wait_for_conditions":
            print(self.tools["wait_for_conditions"]())

        elif action == "report_goal_achieved":
            print(self.tools["report_goal_achieved"]())
            environment.flight_status = "landed" # Ensure environment state is consistent

        elif action == "no_action":
            print(f"{self.name}: No specific action decided by LLM, monitoring.")


    def learn(self, previous_observation, action_taken_data, new_observation):
        """
        Agent's learning module (simplified).
        """
        action_name = action_taken_data[0]
        if new_observation["flight_status"] == "landed" and action_name == "initiate_landing":
            print(f"{self.name}: Learning: Successful landing. Reinforcing this action sequence for future optimal completions.")
        elif action_name == "replan_flight_due_to_conditions" and previous_observation["weather"] == "stormy":
            print(f"{self.name}: Learning: LLM suggested replanning during stormy weather. Validating this strategy.")

        if new_observation["fuel_level"] <= 10 and previous_observation["flight_status"] == "in-flight" and new_observation["flight_status"] != "landed":
            print(f"{self.name}: Learning: High priority on landing when fuel is critically low.")


# --- Test Cases (No Simulation Loop) ---
def run_test_case(name, initial_env_state, steps_to_run):
    print(f"\n--- Running Test Case: {name} ---")
    env = FlightEnvironment(
        initial_weather=initial_env_state.get("weather", "clear"),
        initial_traffic=initial_env_state.get("traffic", "low")
    )
    env.current_location = initial_env_state.get("current_location", "Montreal")
    env.destination = initial_env_state.get("destination", "New York")
    env.flight_status = initial_env_state.get("flight_status", "idle")
    env.fuel_level = initial_env_state.get("fuel_level", 100)
    env.time_elapsed = initial_env_state.get("time_elapsed", 0)

    agent = FlightPlanningAgent(client, MISTRAL_MODEL)

    for i in range(steps_to_run):
        print(f"\n--- Test Step {i + 1} for '{name}' ---")

        # Manually inject environmental changes for specific test steps if needed
        # Note: With steps_to_run=1, these conditional changes will not trigger
        # unless the condition is met at step 1.
        if name == "Stormy Weather Reroute" and i == 5:
            env.weather = "stormy"
            print(f"!!! FORCING WEATHER CHANGE to stormy in environment at step {i+1} !!!")
        if name == "Traffic Delay" and i == 3:
            env.traffic = "high"
            print(f"!!! FORCING TRAFFIC CHANGE to high in environment at step {i+1} !!!")
        if name == "Emergency Landing (Low Fuel)" and i == 2:
            env.fuel_level = 10
            print(f"!!! FORCING FUEL TO CRITICAL (10%) at step {i+1} !!!")


        env.update_environment()

        observation = agent.perceive(env.get_observation())

        print(f"Test case allowing LLM to reason based on current observation for decision making.")
        # Pass the environment object to the reason method
        action_to_take, action_params = agent.reason(observation, env)

        if action_to_take == "report_goal_achieved":
            print(f"\nTEST CASE '{name}' COMPLETE: Goal Achieved by LLM's recommendation!")
            agent.act(("report_goal_achieved", {}), env)
            break # Exit the loop if goal achieved

        print(f"LLM decided action: {action_to_take} with params: {action_params}")
        agent.act((action_to_take, action_params), env)

        agent.learn(observation.copy(), (action_to_take, action_params.copy()), env.get_observation())
        time.sleep(1) # Pause for readability

    print(f"\n--- End of Test Case: {name} ---")
    print(f"Final status for '{name}': {env.flight_status}, Fuel: {env.fuel_level}%, Time: {env.time_elapsed}")


# --- Define Test Cases ---
if __name__ == "__main__":
    # All test cases set to run for 1 step as requested.
    # This will check the LLM's *initial* decision for each scenario.

    # Test Case 1: Ideal Scenario - Plan and Fly
    test_case_1_env = {
        'current_location': 'Montreal',
        'destination': 'New York',
        'weather': 'clear',
        'traffic': 'low',
        'flight_status': 'idle',
        'fuel_level': 100,
        'time_elapsed': 0
    }
    test_case_1_steps = 1 # Set to 1 step for quick testing

    # Test Case 2: Encountering Stormy Weather - Should Replan or Wait
    test_case_2_env = {
        'current_location': 'Montreal',
        'destination': 'New York',
        'weather': 'cloudy', # Starts cloudy - LLM should plan, not wait yet.
        'traffic': 'medium',
        'flight_status': 'idle',
        'fuel_level': 100,
        'time_elapsed': 0
    }
    test_case_2_steps = 1 # Set to 1 step for quick testing

    # Test Case 3: Emergency Landing (Low Fuel) - CRITICAL TEST
    test_case_3_env = {
        'current_location': 'Somewhere_Midflight',
        'destination': 'New York',
        'weather': 'clear',
        'traffic': 'low',
        'flight_status': 'in-flight', # Start already in flight
        'fuel_level': 15, # Set directly to critical fuel for 1-step test
        'time_elapsed': 10
    }
    test_case_3_steps = 1 # Set to 1 step for quick testing

    # Test Case 4: Traffic Delay - Pre-flight (Should wait if traffic is high)
    test_case_4_env = {
        'current_location': 'Montreal',
        'destination': 'New York',
        'weather': 'clear',
        'traffic': 'high', # Set directly to high traffic for 1-step test
        'flight_status': 'planning',
        'fuel_level': 100,
        'time_elapsed': 5
    }
    test_case_4_steps = 1 # Set to 1 step for quick testing

    # Execute Test Cases
    run_test_case("Ideal Flight", test_case_1_env, test_case_1_steps)
    run_test_case("Stormy Weather Reroute", test_case_2_env, test_case_2_steps)
    run_test_case("Emergency Landing (Low Fuel)", test_case_3_env, test_case_3_steps)
    run_test_case("Traffic Delay", test_case_4_env, test_case_4_steps)


--- Running Test Case: Ideal Flight ---

--- Test Step 1 for 'Ideal Flight' ---
Environment Status (Time: 1): Weather: clear, Traffic: low, Fuel: 100%

Mistral_Flight_Agent_2.0_LLM perceives: {'current_location': 'Montreal', 'destination': 'New York', 'weather': 'clear', 'traffic': 'low', 'flight_status': 'idle', 'fuel_level': 100, 'time_elapsed': 1, 'active_flight_plan': None}
Test case allowing LLM to reason based on current observation for decision making.
Mistral_Flight_Agent_2.0_LLM is reasoning using Mistral AI...
LLM Raw Response: {
  "action": "plan_optimal_route",
  "parameters": {
    "current_location": "Montreal",
    "destination": "New York",
    "weather": "clear",
    "traffic": "low"
  },
  "plan_details": {
    "route": "Montreal to New York",
    "distance": "330 miles",
    "estimated_time": "1 hour 30 minutes",
    "waypoints": ["Montreal", "Albany", "New York"],
    "altitude": "35,000 feet",
    "speed": "500 mph"
  }
}
LLM decided action: plan_optimal_route wit